In [1]:
from tools.base import *
from tools.ddmtd import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import sleep 
from copy import deepcopy
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
matplotlib.style.available
matplotlib.style.use(['seaborn-darkgrid'])
# plt.rcParams['figure.figsize'] = [4, 3]
plt.rcParams['figure.dpi'] = 300

/tmp/ipykernel_2492473/4264932307.py:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  matplotlib.style.use(['seaborn-darkgrid'])


In [3]:
freq=160

In [4]:
bin(6)

'0b110'

In [5]:
#Generate column names
NUM_WORDS = 32
column_names = []
for i in range(1,NUM_WORDS+1):
    column_names.append(f"edge{i}")
    column_names.append(f"ddmtd{i}")
    
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', 100)

In [6]:
# freq=160
# board_num = 4
# server="board3_trenz"
# rpi_server="board3_pi"
# !../RPi/config_pll.sh {rpi_server} ../RPi/ 2021_10_05_Si5344H/OCXO/Si5344H-RevD-Registers_{freq}M.h 2021_10_05_Si5344H/PLL/Si5344H-RevD-Registers_{freq}M_100K.h {board_num}

In [7]:
server="kria"


In [8]:
0b11

3

In [9]:
# def get_data(step,channel =5,folder="",aq_iter = 100):
#     data_folder = f"data_files/{folder}/c{channel}_data_step_{str(step).zfill(3)}/"
#     !mkdir -p {data_folder}
#     !../acq_software/run_atPi.sh ../acq_software/ {step} {channel} {rpi_server}
#     !../acq_software/run_atKria.sh ../acq_software/ {aq_iter} {server}
#     !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
#     !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
#     !sshpass -p "root" rsync -aP {server}:data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt


def get_data(fine_delay,coarse_delay ,channel =5,folder="",aq_iter = 10):
    data_folder = f"data_files/{folder}/c{channel}_fstep_{str(fine_delay).zfill(3)}_cstep_{str(coarse_delay).zfill(3)}"
    !mkdir -p {data_folder}
    !../acq_software/run_atKria.sh ../acq_software/ {aq_iter} {server}
    !rsync -aP {server}:data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
    !rsync -aP {server}:data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
    !rsync -aP {server}:data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt
        
        
#def debug_spi(fine_delay,coarse_delay ,channel =5,folder="",aq_iter = 100):
#    data_folder = f"data_files/{folder}/c{channel}_fstep_{str(fine_delay).zfill(3)}_cstep_{str(coarse_delay).zfill(3)}"
#    !mkdir -p {data_folder}
#    !../acq_software/run_atPi.sh ../acq_software/ {fine_delay} {coarse_delay} {channel} {rpi_server}
#     !../acq_software/run_atKria.sh ../acq_software/ {aq_iter} {server}
#     !rsync -aP {server}:data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
#     !rsync -aP {server}:data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
#     !rsync -aP {server}:data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt
        
        
        

def plot_data(fine_delay=0,coarse_delay=0,channel =10,folder="",freq=160*10**6,data_stream = (1,3),draw=False):   
    df = data2df(fine_delay
             ,coarse_delay
             ,channel=channel
             ,folder=folder
             ,freq = freq
             ,data_stream = data_stream
            )
#     popt,p_err = df.drawTIE(save_name='',bns=None,fit=True, sep="TIE_FALL")
    popt,p_err = df.drawTIE(save_name='',bns=None,fit=True, sep="",draw=draw)
    


    return (popt[1]*1000,p_err[1]*1000)
    
    

def plot_tie(tie):
    f,ax = plt.subplots(figsize=(8,6))
    
    ax.errorbar(tie[0],tie[1],yerr=tie[2],fmt='o',
#                         color='red',
                        capsize=5,
                        ecolor='black'
                        ,label=f'ODD'
                       )
#     ax.errorbar(tie[0],tie[3],yerr=tie[4],fmt='o',
# #                         color='red',
#                         capsize=5,
#                         ecolor='black'
#                         ,label=f'EVEN'
#                        )
    plt.xlabel("Delay cell index", fontsize=15)
    plt.ylabel("TIE [ps] staggered", fontsize=15)
    plt.legend(loc="upper right")
    plt.gca().grid()
    
def data2df(fine_delay=0,coarse_delay=0,channel =5,folder="",freq=160*10**6, data_stream = (1,3)):
    data_folder = f"data_files/{folder}/c{channel}_fstep_{str(fine_delay).zfill(3)}_cstep_{str(coarse_delay).zfill(3)}"
    dv1= pd.read_csv(f"{data_folder}/ddmtd1.txt",sep=",",header=0 ,skiprows=0,names=column_names[0:16])
    dv2= pd.read_csv(f"{data_folder}/ddmtd2.txt",sep=",",header=0 ,skiprows=0,names=column_names[16:32])
    dv3= pd.read_csv(f"{data_folder}/ddmtd3.txt",sep=",",header=0 ,skiprows=0,names=column_names[32:48])
    dv = pd.concat((dv1,dv2,dv3),axis=1)  
#     dv = dv1
#     df = deepcopy(dv[(dv.T != 0).any()])
#     df = df.reset_index(drop=1)ore loop

    df = deepcopy(dv)
#     return df 
#     df = df[df.ddmt d3!=2147483647].reset_index(drop=1).drop([0,1,2])    
#     df.iloc[0,:]=0
#     return df
    data = ddmtd(deepcopy(df),q=1,channel=data_stream)
    data.N=100000
    data.INPUT_FREQ = freq
    data.Recalc()
    return data 

#def config_dcps(step,channel):
#    !../acq_software/run_atPi.sh ../acq_software/ {step} {channel} {rpi_server}


#Nishant remove sshpass and use scp -r
#Just use rsync -aP ub
def get_data_debug(step,channel =5,folder="",aq_iter = 100,sleep_time=20000):
    data_folder = f"data_files/{folder}/c{channel}_data_step_{str(step).zfill(3)}/"
    !mkdir -p {data_folder}
    !../acq_software/run_atKria.sh ../acq_software/ {aq_iter} {server} {sleep_time}
    !rsync -aP {server}:~/dcps_ddmtd_toolkit/acq_software/data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
    !rsync -aP {server}:~/dcps_ddmtd_toolkit/acq_software/data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
    !rsync -aP {server}:~/dcps_ddmtd_toolkit/acq_software/data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt
        
        
def copy_data(step,channel =5,folder="",aq_iter = 100,sleep_time=20000):
    data_folder = f"data_files/{folder}/c{channel}_data_step_{str(step).zfill(3)}/"
    !mkdir -p {data_folder}
    !rsync -aP {server}:~/dcps_ddmtd_toolkit/acq_software/data_acq/data_files/ddmtd1.txt ./{data_folder}/ddmtd1.txt
    !rsync -aP {server}:~/dcps_ddmtd_toolkit/acq_software/data_acq/data_files/ddmtd2.txt ./{data_folder}/ddmtd2.txt
    !rsync -aP {server}:~/dcps_ddmtd_toolkit/acq_software/data_acq/data_files/ddmtd3.txt ./{data_folder}/ddmtd3.txt

In [10]:
#Nishant commenting out
#board_num = 4

# for i in range(100):
#!../RPi/config_pll.sh {rpi_server} ../RPi/ 2021_10_05_Si5344/OCXO/Si5344-RevD-Registers_{freq}M.h 2021_10_05_Si5344/PLL/Si5344-RevD-Registers_{freq}M_100K.h {board_num}

# board_num=3
# !../RPi/config_pll.sh {rpi_server} ../RPi/ 2021_10_05_Si5344H/OCXO/Si5344H-RevD-Registers_{freq}M.h 2021_10_05_Si5344H/PLL/Si5344H-RevD-Registers_{freq}M_100K.h {board_num}

In [11]:
#Nishant When running the notebook use the current date
folder_name = f"2023_05_16_{freq}MHz"

In [12]:

# for i in range(100):
#     debug_spi(0,16,channel=4,folder=folder_name,aq_iter=100)
#     sleep(.1)
# #     debug_spi(0,16,channel=5,folder=folder_name,aq_iter=100)
# #     sleep(.1)




In [13]:
# for i in range(6):
#     debug_spi(0,16,channel=i,folder=folder_name,aq_iter=100)
#     sleep(1)

In [14]:
#channel




In [15]:
#Change date to current date of executing the script
# channel = 0
# get_data(0,0,channel=channel,folder=folder_name,aq_iter=10)
channel = 1
get_data(0,0,channel=channel,folder=folder_name,aq_iter=10)
df1 = pd.read_csv(f"{data_save_folder+run_name}_ddmtd1.txt",names=['edge1','ddmtd1'])
df2 = pd.read_csv(f"{data_save_folder+run_name}_ddmtd2.txt",names=['edge2','ddmtd2'])
df = pd.concat((df1,df2),axis=1)

channel = 2
get_data(0,0,channel=channel,folder=folder_name,aq_iter=10)

Copying Data_acq Folder
do_load.sh                                    100%  143   103.5KB/s   00:00    
list-gpio-lines.c                             100% 2168     2.0MB/s   00:00    
compile.sh                                    100%  520   601.0KB/s   00:00    
dma-proxy-test.c                              100% 4480     4.3MB/s   00:00    
get_data_bram.c                               100% 9854     7.0MB/s   00:00    
get_data2.c                                   100% 5263     4.8MB/s   00:00    
dma-proxy.h                                   100%  702   894.8KB/s   00:00    
stream_shared.h                               100%  489   605.6KB/s   00:00    
dma_acq.h                                     100% 2194     2.3MB/s   00:00    
get_data.c                                    100% 3488     3.5MB/s   00:00    
main.hwdef                                    100%  714KB   8.6MB/s   00:00    
main.hwdef                                    100%  714KB  56.0MB/s   00:00    
design_1.bda    

In [16]:
#get_data(24,0,channel=channel,folder=folder_name,aq_iter=100)

In [17]:
# #This requires the actual DDMTD
# X  = []
# Y  = []
# Ye = []
# for f_step in range(0,32):
#     try:
#         tie,tie_err = plot_data(f_step
#              ,0
#              ,channel=channel
#              ,folder=folder_name
#              ,freq = freq*10**6
#              ,data_stream = (2,10)
#              ,draw=False
#                 )
#         X.append(f_step)
#         Y.append(tie)
#         Ye.append(tie_err)
#     except:
#         print(f"{f_step} lost")

In [18]:
# x = np.asarray(X)
# y = np.abs(np.asarray(Y) - Y[0])
# y_err = np.asarray(Ye)
# f,ax = plt.subplots()

# popt,pcov = np.polyfit(x,y,1,cov=True)
# p_e = np.sqrt(np.diag(pcov))

# ax.scatter(x,y)
# ax.plot(x,popt[0]*x+popt[1],color="r",linestyle='--',label=f"Channel 0 \n{popt[0]:4.3}+/- {p_e[0]:4.2} [ps/step]")
# ax.set_ylabel("Delay [ps]")
# ax.set_xlabel("Fine Step")
# ax.legend(loc="upper left",fontsize=10)

In [19]:
# channel = 0
# get_data(0,0,channel=channel,folder=folder_name,aq_iter=100)

In [20]:
#Nishant comment
channel = 0
get_data(0,0,channel=channel,folder=folder_name,aq_iter=10)

Copying Data_acq Folder
do_load.sh                                    100%  143   168.7KB/s   00:00    
list-gpio-lines.c                             100% 2168     2.3MB/s   00:00    
compile.sh                                    100%  520   723.2KB/s   00:00    
dma-proxy-test.c                              100% 4480     4.4MB/s   00:00    
get_data_bram.c                               100% 9854     8.5MB/s   00:00    
get_data2.c                                   100% 5263     3.1MB/s   00:00    
dma-proxy.h                                   100%  702   975.2KB/s   00:00    
stream_shared.h                               100%  489   696.9KB/s   00:00    
dma_acq.h                                     100% 2194     2.6MB/s   00:00    
get_data.c                                    100% 3488     3.9MB/s   00:00    
dma_acq.c                                     100% 7740     4.8MB/s   00:00    
stream_shared.c                               100% 1957     2.2MB/s   00:00    
get_data_driver.

In [21]:
# tie,tie_err = plot_data(0
#                  ,1
#                  ,channel=channel
#                  ,folder=folder_name
#                  ,freq = freq*10**6
#                  ,data_stream = (11,7)
#                  ,draw=True
#                     )

FileNotFoundError: [Errno 2] No such file or directory: 'data_files/2023_05_16_160MHz/c0_fstep_000_cstep_001/ddmtd1.txt'

In [22]:
#folder_name = f"2022_05_01_{freq}MHz"

In [23]:
#get_data(0,7,channel=channel,folder=folder_name,aq_iter=100)

In [ ]:
# X  = []
# Y  = []
# Ye = []
# for c_step in range(0,32):
#     try:
#         tie,tie_err = plot_data(0
#              ,c_step
#              ,channel=channel
#              ,folder=folder_name
#              ,freq = freq*10**6
#              ,data_stream = (2,10)
#              ,draw=False
#                 )
#         X.append(c_step)
#         Y.append(tie)
#         Ye.append(tie_err)
#     except:
#         print(f"{c_step} lost")

In [ ]:
# np.diff(Y)

In [ ]:
# Y

In [ ]:
# x = np.asarray(X)
# y = np.abs(np.asarray(Y) - Y[0])
# y_err = np.asarray(Ye)
# f,ax = plt.subplots()

# popt,pcov = np.polyfit(x,y,1,cov=True)
# p_e = np.sqrt(np.diag(pcov))

# ax.scatter(x,y)
# ax.plot(x,popt[0]*x+popt[1],color="r",linestyle='--',label=f"Channel 0 \n{popt[0]:4.3}+/- {p_e[0]:4.2} [ps/step]")
# ax.set_ylabel("Delay [ps]")
# ax.set_xlabel("Coarse Step")
# ax.legend(loc="upper left",fontsize=10)

In [ ]:

# X  = []
# Y  = []
# Ye = []
# for c_step in (0,1,2,4,8,16):
#     try:
#         tie,tie_err = plot_data(0
#              ,c_step
#              ,channel=channel
#              ,folder=folder_name
#              ,freq = freq*10**6
#              ,data_stream = (2,10)
#              ,draw=False
#                 )
#         X.append(c_step)
#         Y.append(tie)
#         Ye.append(tie_err)
#     except:
#         print(f"{c_step} lost")

In [ ]:
# np.abs(Y-Y[0])

In [ ]:
# data = {}
# # folder_name = f"2022_04_12_{freq}MHz"
# folder_name = f"2022_05_01_{freq}MHz"


# for chan in (3,6,7,10,11):
#     X  = []
#     Y  = []
#     Ye = []
#     for c_step in (0,16):
#         try:
#             tie,tie_err = plot_data(0
#                  ,c_step
#                  ,channel=channel
#                  ,folder=folder_name
#                  ,freq = freq*10**6
#                  ,data_stream = (2,chan)
#                  ,draw=False
#                     )
#             X.append(c_step)
#             Y.append(tie)
#             Ye.append(tie_err)
#         except:
#             print(f"{c_step} lost")
    
#     data[f"chan{chan}"] = np.asarray((X,Y,Ye))
# #2,3
# #6,7
# #10,11


In [ ]:
# !open .


In [25]:
for i in data.keys():
    X = data[i][0]*8
    Y = data[i][1]
#     print(Y)
    try:
        print(f"{i}\t{np.abs(np.diff(Y))[0]:5.5f}ps")
    except:
        print(f"{i}   lost")

NameError: name 'data' is not defined

In [23]:
# f,ax = plt.subplots(figsize=(8,6))
for i in data.keys():
    f,ax = plt.subplots(figsize=(8,6))

    X = data[i][0]*8
    Y = data[i][1]
#     Ye = data[i][2]
    ax.scatter(X,Y)
    
    print(i)
    ax.set_xlabel("ps")

NameError: name 'data' is not defined

In [ ]:
# data["chan3"][1]

In [24]:
X = np.asarray(data).T[0]
Y = np.asarray(data).T[1]
Ye = np.asarray(data).T[2]



plt.scatter(X[X>20],Y[X>20])
plt.scatter(X,Y)

NameError: name 'data' is not defined

In [ ]:
# data = []
# for c_step in range(0,32):
#     try:
#         tie,tie_err = plot_data(0
#                  ,c_step
#                  ,channel=10
#                  ,folder=folder_name
#                  ,freq = freq*10**6
#                  ,data_stream = (2,6)
#                  ,draw=False
#                     )
#         data.append((c_step,tie,tie_err))
#     except:
#         print(f"{c_step} lost")
#2,3
#6,7
#10,11



In [ ]:
# folder_name = f"2022_04_14_{freq}MHz"
# for i in range(0,32):
#     get_data(i,0,channel=10,folder=folder_name,aq_iter=100)